In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime

#Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# sklearn stuff
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, Imputer 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

### Submission Functions 

In [2]:
def generate_submissions(oct_model,nov_model,dec_model,name='new_submission',logy=True):
    """
    This function creates the submission file for the public leaderboard predictions.
    Three already fitted models, one for each of the predicting time points, is required.
    """
    submission_df = DataFrame()
    for i in range(int(properties.shape[0] / 100000)):
        all_feats = full_pipeline.transform(properties.iloc[i*100000:(i+1)*100000])
        foo = properties.iloc[i*100000:(i+1)*100000][['parcelid']].reset_index(drop=True)
        if logy:
            foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                            '201611': nov_model.predict(all_feats),
                                                            '201612': dec_model.predict(all_feats)})], axis=1)
        else:
            foo = pd.concat([foo, DataFrame({'201610': np.log(oct_model.predict(all_feats)),
                                                            '201611': np.log(nov_model.predict(all_feats)),
                                                            '201612': np.log(dec_model.predict(all_feats))})], axis=1)
        submission_df = pd.concat([submission_df, foo], ignore_index=True)

    #  fencepost problem
    all_feats = full_pipeline.transform(properties.iloc[2900000:])
    foo = properties.iloc[2900000:][['parcelid']].reset_index(drop=True)
    foo = pd.concat([foo, DataFrame({'201610': oct_model.predict(all_feats),
                                                    '201611': nov_model.predict(all_feats),
                                                    '201612': dec_model.predict(all_feats)})], axis=1)
    submission_df = pd.concat([submission_df, foo], ignore_index=True)
    
    submission_df['201710'] = 0
    submission_df['201711'] = 0
    submission_df['201712'] = 0
    
    submission_df.rename(columns={'parcelid':'ParcelId'}, inplace=True)    
#     submission_df[['201610','201611','201612','201710','201711','201712']]= submission_df[['201610','201611','201612',
#                                                                                            '201710','201711','201712']].round(4)
    # unit test
    submission_df.drop_duplicates(inplace=True)
    assert submission_df.shape[0] == properties.shape[0]
    # write to .csv
    submission_df[['ParcelId','201610','201611','201612',
                  '201710','201711','201712']].to_csv(name + ".gz", index=False, float_format='%.4g', compression='gzip')
    return submission_df

In [3]:
def mean_absolute_errors(submission_df, comparison_df):
    """
    This function takes a submission entry for public leaderboard, and returns
    the training error for each month.
    """
    # training error
    trainresults = pd.merge(submission_df[['ParcelId','201610','201611','201612']], comparison_df[['parcelid','logerror','month']],
                           left_on='ParcelId', right_on='parcelid')
    oct_error = abs(trainresults[trainresults['month'] == 10]['201610'] 
                    - trainresults[trainresults['month'] == 10]['logerror']).mean()
    nov_error = abs(trainresults[trainresults['month'] == 11]['201611'] 
                    - trainresults[trainresults['month'] == 11]['logerror']).mean()
    dec_error = abs(trainresults[trainresults['month'] == 12]['201612'] 
                    - trainresults[trainresults['month'] == 12]['logerror']).mean()
    overall_mae = (oct_error*(trainresults['month'] == 10).sum() + nov_error*(trainresults['month'] == 11).sum() 
                        + dec_error*(trainresults['month'] == 12).sum()) / (trainresults['month'].isin([10,11,12])).sum()
    return (oct_error, nov_error, dec_error, overall_mae)

### Reading in data 

In [4]:
maindir = "/home/anerdi/Desktop/Zillow"

logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
properties = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv")
test_parcels = pd.read_csv(maindir + "/data/sample_submission.csv", usecols = ['ParcelId'])

test_parcels.rename(columns={'ParcelId':'parcelid'}, inplace=True)

/home/anerdi/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Additional Features

In [5]:
#life of property
properties['N-life'] = 2018 - properties['yearbuilt']

#error in calculation of the finished living area of home
properties['N-LivingAreaError'] = properties['calculatedfinishedsquarefeet']/properties['finishedsquarefeet12']

#proportion of living area
properties['N-LivingAreaProp'] = properties['calculatedfinishedsquarefeet']/properties['lotsizesquarefeet']
properties['N-LivingAreaProp2'] = properties['finishedsquarefeet12']/properties['finishedsquarefeet15']

#Amout of extra space
properties['N-ExtraSpace'] = properties['lotsizesquarefeet'] - properties['calculatedfinishedsquarefeet'] 
properties['N-ExtraSpace-2'] = properties['finishedsquarefeet15'] - properties['finishedsquarefeet12'] 

#Total number of rooms
properties['N-TotalRooms'] = properties['bathroomcnt']*properties['bedroomcnt']

#Average room size
properties['N-AvRoomSize'] = properties['calculatedfinishedsquarefeet']/properties['roomcnt'] 

# Number of Extra rooms
properties['N-ExtraRooms'] = properties['roomcnt'] - properties['N-TotalRooms'] 

#Ratio of the built structure value to land area
properties['N-ValueProp'] = properties['structuretaxvaluedollarcnt']/properties['landtaxvaluedollarcnt']

#Does property have a garage, pool or hot tub and AC?
properties['N-GarPoolAC'] = ((properties['garagecarcnt']>0) & (properties['pooltypeid10']>0) & (properties['airconditioningtypeid']!=5))*1 

properties["N-location"] = properties["latitude"] + properties["longitude"]
properties["N-location-2"] = properties["latitude"]*properties["longitude"]
properties["N-location-2round"] = properties["N-location-2"].round(-4)

properties["N-latitude-round"] = properties["latitude"].round(-4)
properties["N-longitude-round"] = properties["longitude"].round(-4)

#Ratio of tax of property over parcel
properties['N-ValueRatio'] = properties['taxvaluedollarcnt']/properties['taxamount']

#TotalTaxScore
properties['N-TaxScore'] = properties['taxvaluedollarcnt']*properties['taxamount']

#polnomials of tax delinquency year
properties["N-taxdelinquencyyear-2"] = properties["taxdelinquencyyear"] ** 2
properties["N-taxdelinquencyyear-3"] = properties["taxdelinquencyyear"] ** 3

#Length of time since unpaid taxes
properties['N-life'] = 2018 - properties['taxdelinquencyyear']

#Number of properties in the zip
zip_count = properties['regionidzip'].value_counts().to_dict()
properties['N-zip_count'] = properties['regionidzip'].map(zip_count)

#Number of properties in the city
city_count = properties['regionidcity'].value_counts().to_dict()
properties['N-city_count'] = properties['regionidcity'].map(city_count)

#Number of properties in the city
region_count = properties['regionidcounty'].value_counts().to_dict()
properties['N-county_count'] = properties['regionidcounty'].map(region_count)

#Average structuretaxvaluedollarcnt by city
group = properties.groupby('regionidcity')['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
properties['N-Avg-structuretaxvaluedollarcnt'] = properties['regionidcity'].map(group)

#Deviation away from average
properties['N-Dev-structuretaxvaluedollarcnt'] = (abs((properties['structuretaxvaluedollarcnt'] 
                                                       - properties['N-Avg-structuretaxvaluedollarcnt']))
                                                  /properties['N-Avg-structuretaxvaluedollarcnt'])

In [6]:
# join on parcel id
data = pd.merge(properties,logerror[['parcelid','logerror','month']], on='parcelid')

### Data  Preprocessing Pipeline

In [7]:
# Setup variables considered in the model

# numerical variables
num_atts = ['bathroomcnt','bedroomcnt','buildingqualitytypeid','calculatedbathnbr','finishedfloor1squarefeet',
           'calculatedfinishedsquarefeet','finishedsquarefeet12','finishedsquarefeet13',
           'finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6','fireplacecnt',
           'fullbathcnt','garagecarcnt','garagetotalsqft','latitude','longitude','lotsizesquarefeet',
           'poolcnt','poolsizesum','censustractandblock','roomcnt','threequarterbathnbr','unitcnt',
           'yardbuildingsqft17','yardbuildingsqft26','numberofstories',
            'structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount',
           'N-ValueRatio', 'N-LivingAreaProp', 'N-ValueProp', 'N-Dev-structuretaxvaluedollarcnt', 
            'N-TaxScore', 'N-zip_count', 'N-Avg-structuretaxvaluedollarcnt', 'N-city_count',
           'N-LivingAreaProp2', 'N-location-2round', 'N-TotalRooms','N-AvRoomSize']

# categorical varaibles
cat_atts = ['airconditioningtypeid','architecturalstyletypeid',
           'buildingclasstypeid','heatingorsystemtypeid','pooltypeid10','pooltypeid2',
            'pooltypeid7','propertylandusetypeid','regionidcounty',
           'storytypeid','typeconstructiontypeid','yearbuilt','fireplaceflag',
           'taxdelinquencyflag']

# Dictionary of categorical variables and their default levels
cat_dict = {'airconditioningtypeid':[-1] + list(range(1,14)),
           'architecturalstyletypeid':[-1] + list(range(1,28)),
           'buildingclasstypeid':[-1] + list(range(1,6)),
            'heatingorsystemtypeid':[-1] + list(range(1,26)),
            'pooltypeid10': list(range(-1,2)),
            'pooltypeid2': list(range(-1,2)),
            'pooltypeid7': list(range(-1,2)),
            'propertylandusetypeid': [-1, 31,46,47,246,247,248,260,261,262,263,264,265,266,267,268,269,270,271,
                                     273,274,275,276,279,290,291],
            'regionidcounty': [2061,3101,1286],
            'storytypeid':[-1] + list(range(1,36)),
            'typeconstructiontypeid':[-1] + list(range(1,19)),
            'yearbuilt': [-1] + list(range(1885,2018)),
            'fireplaceflag': [-1] + ['True','False'],
            'taxdelinquencyflag': [-1] + ['Y','N']
           }

In [8]:
# A custom transformer, which selects certain variables
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, desired_cols):
        self.desired_cols = desired_cols
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.desired_cols].values

# A custom transformer, which first selects the categorical variables
# from the DataFrame and then performs the dummification
class DF_Selector_GetDummies(BaseEstimator, TransformerMixin):
    def __init__(self, cat_dict):
        self.cat_dict = cat_dict
        self.ndummies = sum(len(c) - 1  for c in cat_dict.values()) 
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.fillna(-1) # missing values are given -1 missing label
        foo = np.zeros((X.shape[0],self.ndummies))
        start = 0
        end = 0
        for c in sorted(self.cat_dict.keys()):
            end += len(self.cat_dict[c]) - 1
            foo[:, start:end] = pd.get_dummies(X[c].astype('category', categories=self.cat_dict[c]))[self.cat_dict[c][1:]]
            start += len(self.cat_dict[c]) - 1
        return foo

In [9]:
# Categorical pipeline
cat_pipeline = Pipeline([
        ('select_and_dummify', DF_Selector_GetDummies(cat_dict)),
    ])

# Numerical pipeline
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_atts)),
        ('imputer', Imputer()),
    ])

# Full pipeline
full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline)
    ])

In [10]:
import gc

In [11]:
gc.collect()

28

## Training XGB

In [12]:
import xgboost as xgb
from xgboost import XGBRegressor

In [13]:
data['wts_oct'] = np.where(data['month'] == 10, 1.25, 1)
data['wts_nov'] = np.where(data['month'] == 11, 1.25, 1)
data['wts_dec'] = np.where(data['month'] == 12, 1.25, 1)

## Creating a train/test set

In [16]:
X_train = full_pipeline.fit_transform(data)

In [17]:
X_train.shape

(90275, 334)

In [18]:
Y_train = data['logerror'].values

In [20]:
p75 = np.percentile(data['logerror'],75)
p25 = np.percentile(data['logerror'],25)
outerquartile = data[(data['logerror'] >= p75) | (data['logerror'] <= p25)]
X_train_extreme = full_pipeline.transform(outerquartile)
Y_train_extreme = outerquartile['logerror'].values

In [22]:
oct_xgb = XGBRegressor(random_state=42, n_estimators=600, max_depth=4, learning_rate=0.02,
                          subsample= 1, colsample_bytree= 1)
nov_xgb = XGBRegressor(random_state=42, n_estimators=600, max_depth=4, learning_rate=0.02,
                      subsample= 1, colsample_bytree= 1)
dec_xgb = XGBRegressor(random_state=42, n_estimators=600, max_depth=4, learning_rate=0.02,
                      subsample= 1, colsample_bytree= 1)

oct_extreme_xgb = XGBRegressor(random_state=42, n_estimators=500, max_depth=4, learning_rate=0.02,
                          subsample= 0.7, colsample_bytree= 0.7)
nov_extreme_xgb = XGBRegressor(random_state=42, n_estimators=500, max_depth=4, learning_rate=0.02,
                      subsample= 0.7, colsample_bytree= 0.7)
dec_extreme_xgb = XGBRegressor(random_state=42, n_estimators=500, max_depth=4, learning_rate=0.02,
                      subsample= 0.7, colsample_bytree= 0.7)

In [25]:
print("Training")
print("training oct model...")
oct_xgb.fit(X_train,Y_train,sample_weight=data['wts_oct'], eval_metric='mae')
print("training nov model...")
nov_xgb.fit(X_train,Y_train,sample_weight=data['wts_nov'], eval_metric='mae')
print("training dec model...")
dec_xgb.fit(X_train,Y_train,sample_weight=data['wts_dec'], eval_metric='mae')

submission_df = generate_submissions(oct_xgb, nov_xgb, dec_xgb, name="XGB_600")

Training
training oct model...
training nov model...
training dec model...


In [24]:
print("Training Extreme Data")
print("training oct model...")
oct_extreme_xgb.fit(X_train_extreme,Y_train_extreme,sample_weight=outerquartile['wts_oct'], eval_metric='mae')
print("training nov model...")
nov_extreme_xgb.fit(X_train_extreme,Y_train_extreme,sample_weight=outerquartile['wts_nov'], eval_metric='mae')
print("training dec model...")
dec_extreme_xgb.fit(X_train_extreme,Y_train_extreme,sample_weight=outerquartile['wts_dec'], eval_metric='mae')

extreme_df = generate_submissions(oct_extreme_rf, nov_extreme_rf, dec_extreme_rf)

In [30]:
# test error
mean_absolute_errors(submission_df,data)

(0.06211728041279804,
 0.0613652784356152,
 0.07414280260781426,
 0.06440471046276261)

In [26]:
submission_df.describe()

,ParcelId,201610,201611,201612,201710,201711,201712
count,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2985217.0,2985217.0,2985217.0
mean,1.332586e+07,1.904703e-02,1.890481e-02,1.896906e-02,0.0,0.0,0.0
std,7.909966e+06,4.374892e-02,4.316232e-02,4.358434e-02,0.0,0.0,0.0
min,1.071172e+07,-1.895638e+00,-1.888715e+00,-1.928589e+00,0.0,0.0,0.0
25%,1.164371e+07,6.539196e-03,6.434023e-03,6.405234e-03,0.0,0.0,0.0
50%,1.254509e+07,1.530969e-02,1.527369e-02,1.532555e-02,0.0,0.0,0.0
75%,1.409712e+07,2.694345e-02,2.675608e-02,2.693507e-02,0.0,0.0,0.0
max,1.696019e+08,3.359656e+00,3.340071e+00,3.362306e+00,0.0,0.0,0.0


## Model Averaging 

In [28]:
RF_1 = pd.read_csv("RF_n100_maxfeat5_maxdepth8.gz", compression="gzip")
mean_absolute_errors(RF_1, data)

(0.0627264215390796,
 0.06182163198247547,
 0.07473668775158138,
 0.064978084757667934)

In [34]:
blend1 = submission_df[['ParcelId']].copy()
cols = ['201610','201611','201612','201710','201711','201712']
blend1 = pd.concat([blend1,(5/10)*submission_df[cols] + (5/10)*RF_1[cols]], axis=1)
blend1['ParcelId'] = blend1['ParcelId'].astype(int)
assert all(blend1.ParcelId.unique() == submission_df.ParcelId.unique())
mean_absolute_errors(blend1, data)

(0.062230058315251933,
 0.061397585900942186,
 0.07423508067602663,
 0.06449611301635913)

In [40]:
blend1.to_csv("XGB600_RF100_addfeats.gz", index=False, float_format='%.4g', compression='gzip')

In [29]:
RF_2 = pd.read_csv("RF_n100_maxfeat10_maxdepth20_extreme.gz", compression="gzip")
mean_absolute_errors(RF_2, data)

(0.06195625879043588,
 0.06436095290251921,
 0.07321730879815982,
 0.064762854132521586)

In [37]:
blend2 = submission_df[['ParcelId']].copy()
cols = ['201610','201611','201612','201710','201711','201712']
blend2 = pd.concat([blend2,(9.5/20)*submission_df[cols] + (9.5/20)*RF_1[cols] + (1/20)*RF_2[cols]], axis=1)
blend2['ParcelId'] = blend2['ParcelId'].astype(int)
assert all(blend2.ParcelId.unique() == submission_df.ParcelId.unique())
mean_absolute_errors(blend2, data)

(0.06191102362731224,
 0.06120129816943749,
 0.07379994397099514,
 0.064179681294320584)

In [ ]:
blend2.to_csv("XGB600_RF100_RFExtreme.gz", index=False, float_format='%.4g', compression='gzip')

In [41]:
XGB3000 = pd.read_csv("XGB_3000.csv")

In [54]:
blend3 = submission_df[['ParcelId']].copy()
cols = ['201610','201611','201612','201710','201711','201712']
blend3 = pd.concat([blend3,(49/100)*submission_df[cols] + (49/100)*RF_1[cols] + (1/100)*XGB3000[cols] + (1/100)*RF_2[cols]], axis=1)
blend3['ParcelId'] = blend3['ParcelId'].astype(int)
assert all(blend3.ParcelId.unique() == submission_df.ParcelId.unique())
mean_absolute_errors(blend3, data)

(0.06212876220113725,
 0.06131796718466208,
 0.07411259274683836,
 0.064395136541911133)

In [53]:
blend3.to_csv("XGB600_RF100_XGB3000_RFext.gz", index=False, float_format='%.4g', compression='gzip')

In [43]:
amyhighscore = pd.read_csv("9010_XGB_3000_RF.gz", compression="gzip")
mean_absolute_errors(amyhighscore, data)

(0.06221434396363281,
 0.06144365500547652,
 0.07391738266532494,
 0.064432127417700866)